In [1]:
import pandas as pd
from datetime import datetime

In [2]:
def read_data (filename, header):
    df=pd.read_csv(filename, header = None, sep=",",names=header)
    for col in df.columns:
        if df[col].dtype=='O':
            df[col]=df[col].apply(lambda x: x.strip('('))
            df[col]=df[col].apply(lambda x: x.strip(')'))
            df[col]=df[col].apply(lambda x: x.strip('"'))
    return df


# all

In [3]:
all_events=read_data('all_events.txt',['eventType', 'uid', 'ip', 'timestamp', 'impressionId'])
print("number of all events: ", len(all_events))

number of all events:  5589


In [4]:
all_events.sort_values(by="timestamp",inplace=True)

In [5]:
all_events.head(3)

,eventType,uid,ip,timestamp,impressionId
0,display,3a0ce0f4-fb33-45ff-8b5a-c8c2188c1f7d,197.117.214.65,1593784721,98047cd5-8796-4b6a-a419-66b126fabfba
1184,click,724a6db9-6731-4d88-aac1-f8f24ee1b48314,238.186.83.58,1593784721,b15f9c2b-140d-4c1c-b821-bdbc7f69abfa
1185,click,3603ee6a-a570-4e52-b8a7-debd44e0b60915,238.186.83.58,1593784721,d07e1be6-b592-4646-9041-bf523594a847


In [6]:
from datetime import datetime

timestamp_1 = int(all_events.iloc[0,3])
timestamp_2 = int(all_events.iloc[-1,3])
firstTimestamp = datetime.fromtimestamp(timestamp_1)
lastTimestamp= datetime.fromtimestamp(timestamp_2)


print("First timestamp =", firstTimestamp)
print("Last timestamp =", lastTimestamp)

print ("stream duration: " , (timestamp_2-timestamp_1)/60 , "minutes")

First timestamp = 2020-07-03 15:58:41
Last timestamp = 2020-07-03 16:03:32
stream duration:  4.85 minutes


# Pattern 1



In [7]:
df_pattern1=read_data('Pattern_1.txt',['ip', "uid_count"] )

In [8]:
df_pattern1.head()

,ip,uid_count
0,238.186.83.58,17.0
1,173.108.112.51,3.0
2,123.238.127.219,5.0
3,199.255.202.237,4.0
4,220.177.45.132,3.0


#### Suspicious events using uid

In [9]:
events_pattern1=all_events[all_events["ip"].isin(df_pattern1['ip'].values) ]

In [10]:
events_pattern1.head()

,eventType,uid,ip,timestamp,impressionId
1184,click,724a6db9-6731-4d88-aac1-f8f24ee1b48314,238.186.83.58,1593784721,b15f9c2b-140d-4c1c-b821-bdbc7f69abfa
1185,click,3603ee6a-a570-4e52-b8a7-debd44e0b60915,238.186.83.58,1593784721,d07e1be6-b592-4646-9041-bf523594a847
1186,click,9b4cab60-8aa4-45d6-9beb-65e2616508e316,238.186.83.58,1593784721,95ecd404-13bc-41f4-98ba-dbe51f68148a
1187,click,dc8e61f4-b2ee-45ce-938f-dff115dfd67817,238.186.83.58,1593784721,479b00d0-0d90-4ae6-87a1-84c415531dbf
132,display,69163476-567e-40b1-ab2d-f87816ba773f,48.58.70.246,1593784721,4f3432d8-7135-4a02-8cf6-10eeac392e10


#### Count of fraudulent events 

In [11]:
print("number of suspicious events using uid: ", len(events_pattern1), "  which represents: " , round(len(events_pattern1)/len(all_events)*100,2) , "%")

number of suspicious events using uid:  822   which represents:  14.71 %


# Pattern 2

In [12]:
df_pattern2=read_data('Pattern_2.txt',['uid', "IP_count"] )

In [13]:
df_pattern2.head()

,uid,IP_count
0,83cfcbdb-4f57-4c8d-84e6-6eb08689b71f,4.0
1,9d008e5b-e698-4e15-a588-5266e27a426a,3.0
2,bfa6b92f-8575-45ae-8e43-e55ef28e4516,2.0
3,7e86def1-79f4-48f9-b1d0-d2f1d512917a,2.0
4,69163476-567e-40b1-ab2d-f87816ba773f,3.0


#### Suspicious events using IP address 

In [14]:
events_pattern2=all_events[all_events["uid"].isin(df_pattern2['uid'].values) ]

In [15]:
events_pattern2.head()

,eventType,uid,ip,timestamp,impressionId
0,display,3a0ce0f4-fb33-45ff-8b5a-c8c2188c1f7d,197.117.214.65,1593784721,98047cd5-8796-4b6a-a419-66b126fabfba
132,display,69163476-567e-40b1-ab2d-f87816ba773f,48.58.70.246,1593784721,4f3432d8-7135-4a02-8cf6-10eeac392e10
42,display,8d686f56-e58f-4d32-8f59-98822ad0997d,107.22.30.81,1593784721,be99e4bd-eebd-435a-8f00-38b38a283f83
131,display,9fb7d887-abde-43fe-84d4-65ae6fd1a692,217.252.241.14,1593784721,bf26ab85-e8fc-48c4-bbaf-fea767ab80a1
26,display,13391b2a-3ef2-4a51-8c9c-5b69496cebc5,40.150.198.199,1593784721,635062ba-9b30-48e0-8638-a999f2a421ac



#### Count of fraudulent events 


In [16]:

print("number of suspicious events using uid: ", len(events_pattern2), "  which represents: " , round(len(events_pattern2)/len(all_events)*100,2) , "%")

number of suspicious events using uid:  4264   which represents:  76.29 %


# Pattern 3

In [17]:
df_pattern3=read_data('Pattern_3.txt',['uid',"eventType_clicks", "clicks_count","eventType_displays","displays_count", "CTR"] )

In [18]:
df_pattern3.head()

,uid,eventType_clicks,clicks_count,eventType_displays,displays_count,CTR
0,7e86def1-79f4-48f9-b1d0-d2f1d512917a,click,5.0,display,5.0,1.0
1,93ecae95-b510-459b-8460-ad2fc22a570e,click,8.0,display,12.0,0.6666666666666666
2,d96d864f-1656-4ba0-8ed1-ca53f73cb002,click,4.0,display,4.0,1.0
3,f1546e28-3a17-4dfd-a767-036e4d514394,click,3.0,display,5.0,0.6
4,d976ae8b-d16d-4907-86c0-2057a107ef81,click,8.0,display,4.0,2.0


#### Suspicious events with a suspicious CTR 

In [19]:
events_pattern3=all_events[all_events["uid"].isin(df_pattern3['uid'].values) ]

In [20]:
events_pattern3.head()

,eventType,uid,ip,timestamp,impressionId
26,display,13391b2a-3ef2-4a51-8c9c-5b69496cebc5,40.150.198.199,1593784721,635062ba-9b30-48e0-8638-a999f2a421ac
1190,click,d976ae8b-d16d-4907-86c0-2057a107ef81,227.50.143.57,1593784721,49c89e4c-8a78-4ab3-a9b3-e8c09667b8d4
1191,click,83cfcbdb-4f57-4c8d-84e6-6eb08689b71f,226.237.73.38,1593784721,418aca17-b504-4208-a79c-0d9b55e12540
77,display,40a1a99e-7472-431c-ad32-8ef85060b179,198.240.24.62,1593784721,046958cf-3855-4808-b43e-1dc089fab701
78,display,9d895d87-347b-4e75-b819-71d847dd664b,220.177.45.132,1593784721,a6338b6d-4cb8-458e-b711-8b4231bb2771




#### Count of fraudulent events 

In [21]:
print("number of suspicious events using uid: ", len(events_pattern3), "  which represents: " , round(len(events_pattern3)/len(all_events)*100,2) , "%")

number of suspicious events using uid:  2003   which represents:  35.84 %
